<a href="https://colab.research.google.com/github/pcpiscator/2T2021/blob/main/Furg_ECD_Machine_Learning_II_Semana_04_Detec%C3%A7%C3%A3o_de_anomalias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso de Especialização em Ciência de Dados - FURG
## Machine Learning I - Detecção de anomalias
### Prof. Marcelo Malheiros

Parte do código adaptada de Aurélien Geron (licença Apache-2.0)

---

# Inicialização

Aqui importamos as bibliotecas fundamentais de Python para este _notebook_:

- NumPy: suporte a vetores, matrizes e operações de Álgebra Linear
- Matplotlib: biblioteca de visualização de dados
- Scikit-Learn: biblioteca com algoritmos de Machine Learning

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sklearn

# Dataset problemático

Aqui voltamos ao _dataset_ problemático em que tanto o algoritmo K-Means como o DBSCAN falharam anteriormente. Os problemas aqui são dois:

1. Os dados se organizam em clusters aproximadamente circulares, o que faz com que o K-Means não funcione bem.

2. Os dados apresentam densidade continuamente variável, o que faz com que o DBSCAN não consiga segmentar os diversos _clusters_. Ou todas as instância se juntam em um grande _cluster_, ou muitos pequenos grupos são criados.

In [ ]:
from sklearn.datasets import make_blobs

X1, y1 = make_blobs(n_samples=1000, centers=((4, -4), (0, 0)), random_state=42)
X1 = X1.dot(np.array([[0.374, 0.95], [0.732, 0.598]]))
X2, y2 = make_blobs(n_samples=250, centers=1, random_state=42)
X2 = X2 + [6, -8]
X = np.r_[X1, X2]
y = np.r_[y1, y2]

In [ ]:
def plot_clusters(X, y=None):
    plt.figure(figsize=(8, 4))
    plt.scatter(X[:, 0], X[:, 1], c=y, s=1)
    plt.xlabel('$x_1$', fontsize=14)
    plt.ylabel('$x_2$', fontsize=14, rotation=0)
    plt.show()
    
plot_clusters(X)

# Misturas Gaussianas

Vamos então treinar um modelo de Misturas Gaussianas sobre o _dataset_ anterior.

A biblioteca Sciki-Learn provê o algoritmo `GaussianMixture`, que infere os parâmetros de cada um dos $k$ _clusters_ solicitados pelo analista, via hiperparâmetro `n_components=k`.

**Atenção:** Como o _default_ para o algoritmo `GaussianMixture` é fazer apenas uma tentativa, há o risco de convergir para uma solução ruim. Por isso definimos também o hiperparâmetro `n_init` para fazer 10 tentativas e manter a melhor delas.

In [ ]:
from sklearn.mixture import GaussianMixture

# criação do modelo
gm = GaussianMixture(random_state=42, n_components=3, n_init=10)

# treinamento
gm.fit(X);

Os _clusters_ são definidos pelos seguintes parâmetros: seus pesos (disponíveis em `.weights_`), suas médias (uma para cada dimensão, disponíveis em `.means_`) e respectivas matrizes de covariância (disponíveis em `.covariances_`).

Se formos examinar os valores, todos os três conjuntos de parâmetros se aproximam bastante dos valores usados para gerar esse _dataset_ sintético. Por exemplo, a quantidade de pontos em cada um dos três clusters era de 500, 500 e 250, ou seja, 40%, 40% e mais 20% das instâncias, respectivamente. Esses valores se refletem os pesos encontrados.

In [ ]:
print('pesos:', gm.weights_)

In [ ]:
print('médias:', gm.means_, sep='\n')

In [ ]:
print('matrizes de covariância:', gm.covariances_, sep='\n')

Podemos verificar se o algoritmo de fato convergiu para uma solução examinando o valor booleano `.converged_`.

In [ ]:
print('convergiu?', gm.converged_)

Também podemos checar quantas iterações levou para chegar na solução com `.n_iter_`.

In [ ]:
print('iterações:', gm.n_iter_)

Agora podemos usar o modelo para prever a qual _cluster_ cada instância pertence (usando a ideia de _hard clustering_), usando o método `.predict()`:

In [ ]:
print('clusters:', gm.predict(X))

Ainda podemos estimar as probabilidades de que cada instância tenha originado de cada um dos três _clusters_, chamando o método `predict_proba()`. Isso equivale a fazer _soft clustering_:

In [ ]:
print('probabilidades:', gm.predict_proba(X), sep='\n')

Este modelo é **generativo**, então possibilita a **síntese de novas instâncias** pertencentes a este _dataset_ com `.sample()`, incluindo os respectivos rótulos. Este é um processo de **amostragem** da distribuição probabilística aprendida:

In [ ]:
# cria seis novas instâncias
X_new, y_new = gm.sample(6)
print('novas instâncias:', X_new, sep='\n')

In [ ]:
print('rótulos das novas instâncias:', y_new)

In [ ]:
# ilustração das novas instâncias
plt.figure(figsize=(8, 4))
plt.scatter(X[:, 0], X[:, 1], s=1, c='blue')
plt.scatter(X_new[:, 0], X_new[:, 1], s=20, c='red')
plt.xlabel('$x_1$', fontsize=14)
plt.ylabel('$x_2$', fontsize=14, rotation=0)
plt.show()

Também é possível estimar a densidade do modelo em qualquer local. Isso é obtido usando o método `.score_samples()`. Para qualquer instância dada (original ou sintética), o algoritmo de Mistura Gaussiana estima o _log_ da função de **densidade de probabilidade** (_probability density function_ ou PDF) naquele local. Não confunda com o conceito de probabilidade: para a densidade, valores maiores que um ou negativos são permitidos também.

Quanto maior for a pontuação, maior será a densidade.

In [ ]:
print('densidades:', gm.score_samples(X_new))

In [ ]:
# funções auxiliares para desenho

from matplotlib.colors import LogNorm

def plot_centroids(centroids, weights=None, circle_color='w', cross_color='k'):
    if weights is not None:
        centroids = centroids[weights > weights.max() / 10]
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='o', s=35, linewidths=8,
                color=circle_color, zorder=10, alpha=0.9)
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=2, linewidths=12,
                color=cross_color, zorder=11, alpha=1)

def plot_gaussian_mixture(clusterer, X, resolution=1000, show_ylabels=True):
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    Z = -clusterer.score_samples(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, norm=LogNorm(vmin=1.0, vmax=30.0), levels=np.logspace(0, 2, 12))
    plt.contour(xx, yy, Z, norm=LogNorm(vmin=1.0, vmax=30.0), levels=np.logspace(0, 2, 12),
                linewidths=1, colors='k')
    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    C = plt.contour(xx, yy, Z, linewidths=2, colors='r')
    for c in C.collections:
        c.set_dashes([(0, (2.0, 2.0))])
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)
    plot_centroids(clusterer.means_, clusterer.weights_)
    plt.xlabel('$x_1$', fontsize=14)
    if show_ylabels:
        plt.ylabel('$x_2$', fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)

Agora vamos representar graficamente as fronteiras de decisão resultantes (usando linhas tracejadas) e contornos de densidade de todo o espaço:

In [ ]:
# gráfico
plt.figure(figsize=(8, 4))
plot_gaussian_mixture(gm, X)
plt.show()

Em uma situação geral, pode ser necessário reduzir a dificuldade da tarefa, limitando o número de parâmetros que o algoritmo precisa aprender.

Uma maneira de fazer isso é limitar as formas e orientações que os _clusters_ podem ter. Isso é feito impondo restrições às matrizes de covariância, definindo o hiperparâmetro `covariance_type` para um dos seguintes valores:

- `'full'` (padrão): sem restrição, quando todos os _clusters_ podem assumir qualquer forma elipsoidal, de qualquer tamanho.

- `'tied'`: todos os _clusters_ devem ter a mesma forma, que pode ser qualquer elipsóide (ou seja, todos eles compartilham a mesma matriz de covariância).

- `'spherical'`: todos os _clusters_ devem ser esféricos, mas podem ter diâmetros diferentes (ou seja, variâncias diferentes).

- `'diag'`: os _clusters_ podem assumir qualquer forma elipsoidal de qualquer tamanho, mas os eixos do elipsóide devem ser paralelos aos eixos principais do espaço (ou seja, as matrizes de covariância devem ser diagonais).

In [ ]:
# modelos com diferentes restrições
gm_full = GaussianMixture(random_state=42, n_components=3, n_init=10, covariance_type='full')
gm_tied = GaussianMixture(random_state=42, n_components=3, n_init=10, covariance_type='tied')
gm_sphe = GaussianMixture(random_state=42, n_components=3, n_init=10, covariance_type='spherical')
gm_diag = GaussianMixture(random_state=42, n_components=3, n_init=10, covariance_type='diag')

gm_full.fit(X)
gm_tied.fit(X)
gm_sphe.fit(X)
gm_diag.fit(X);

In [ ]:
# função auxiliar
def compare_gaussian_mixtures(gm1, gm2, X):
    plt.figure(figsize=(12, 4))
    plt.subplot(121)
    plot_gaussian_mixture(gm1, X)
    plt.title("covariance_type='{}'".format(gm1.covariance_type), fontsize=14)
    plt.subplot(122)
    plot_gaussian_mixture(gm2, X, show_ylabels=False)
    plt.title("covariance_type='{}'".format(gm2.covariance_type), fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
compare_gaussian_mixtures(gm_full, gm_tied, X)

In [ ]:
compare_gaussian_mixtures(gm_sphe, gm_diag, X)

## Deteção de anomalias

Misturas gaussianas podem ser usadas para detecção de anomalias: instâncias localizadas em regiões de baixa densidade podem ser consideradas _outliers_.

É preciso definir qual limiar de densidade se deseja. Este limiar pode ser definido como um valor absoluto fixo.

Porém, é mais conveniente definir o valor com base em um percentil, como por exemplo o valor de densidade que marca como _outliers_ apenas 4% das instâncias, como mostrado abaixo:

In [ ]:
# obtém a densidade de todas as instâncias
densidades = gm.score_samples(X)

# identifica o limiar para 4% das amostras em regiões menos densas
limiar = np.percentile(densidades, 4.0)

# seleciona apenas as instâncias com densidade inferior ao limiar
anomalias = X[densidades < limiar]
normais = X[densidades >= limiar]

In [ ]:
# gráfico
plt.figure(figsize=(8, 4))
plot_gaussian_mixture(gm, X)
plt.scatter(anomalias[:, 0], anomalias[:, 1], color='r', marker='x')
plt.show()

Note que modelos como este de Mistura Gaussiana tentam ajustar todos os dados, incluindo os _outliers_. Então, se houver muitos deles, isso pode afetar a definição do que é "normal" pelo modelo, fazendo que alguns valores discrepantes ainda sejam rotulados como normais.

**Dica:** Pode ser útil, então ajustar o modelo a primeira vez, usando este apenas para detectar e remover os _outliers_ mais extremos. Em seguida, um novo ajuste de modelo é feito, para um conjunto de dados mais limpo.

## Seleção de hiperparâmetros

Com K-Means, podíamos usar a inércia ou a pontuação da silhueta para selecionar o número apropriado de _clusters_.  já com Misturas Gaussianas essas métricas não podem mais ser usadas, pois os _clusters_ não são esféricos ou têm
diferentes tamanhos.

Em vez disso, podemos escolher ajustes que minimizam as métricas **Bayesian Information Criterion (BIC)** ou **Akaike Information Criterion (AIC)**.

Tanto a BIC quanto a AIC penalizam modelos que têm mais parâmetros para aprender (por exemplo, com mais _clusters_) e dão mais importância a modelos que se ajustam bem aos dados.

Normalmente ambas as métricas acabam selecionando o mesmo modelo. Quando elas diferem, o modelo selecionado pelo BIC tende a ter misturas gaussianas mais simples (com menos parâmetros) do que aquele selecionado pelo AIC, mas que tende a não ajustar os dados tão bem também.

In [ ]:
# métrica Bayesian Information Criterion
gm.bic(X)

In [ ]:
# métrica Akaike Information Criterion
gm.aic(X)

Como fizemos para o K-Means, podemos treinar uma série de modelos apenas variando o hiperparâmetr $k$, medindo as duas métricas paa cada um deles:

In [ ]:
# treina vários modelos, com k variando de 1 até 10
gm_k = [GaussianMixture(random_state=42, n_components=k, n_init=10).fit(X) for k in range(1, 11)]

In [ ]:
# calcula as métricas BIC e AIC para cada modelo
bics = [model.bic(X) for model in gm_k]
aics = [model.aic(X) for model in gm_k]

In [ ]:
# exibe o gráfico da variação das métricas em função de k
plt.figure(figsize=(8, 4))
plt.plot(range(1, 11), bics, 'bo-', label='BIC')
plt.plot(range(1, 11), aics, 'go--', label='AIC')
plt.xlabel('$k$', fontsize=14)
plt.ylabel('métrica', fontsize=14)
plt.axis([1, 9.5, np.min(aics) - 50, np.max(aics) + 50])
plt.annotate('mínimo', xy=(3, bics[2]), xytext=(0.35, 0.6), textcoords='figure fraction',
             fontsize=14, arrowprops=dict(facecolor='black', shrink=0.1))
plt.legend()
plt.show()

O procedimento anterior apenas permite encontrar o valor mais adequado do hiperparâmetro `n_components`, para o caso mais geral de formatos dos clusters. Ou seja, usando o valor padrão `full` do hiperparâmetro `covariance_type`.

Uma busca mais ampla poderia tambeḿ ser feita, procurando a melhor combinação tanto de $k$ como do formato dos _clusters_.

In [ ]:
# procura pelo melhor par de hiperparâmetros 'n_components' e 'covariance_type'

min_bic = np.infty
for covariance_type in ('full', 'tied', 'spherical', 'diag'):
    print(f"testando tipo de covariância '{covariance_type}' com k igual a", end=' ')
    for k in range(1, 11):
        print(k, end=' ')
        bic = GaussianMixture(random_state=42, n_init=10, n_components=k, 
                              covariance_type=covariance_type).fit(X).bic(X)
        if bic < min_bic:
            min_bic = bic
            best_k = k
            best_covariance_type = covariance_type
    print()

In [ ]:
print('melhor n_components:', best_k)

In [ ]:
print('melhor covariance_type:', best_covariance_type)

## Aprendizado Bayesiano para Misturas Gaussianas 

Em vez de procurar manualmente pelo número ideal de _clusters_, é possível usar a classe `BayesianGaussianMixture`, que define pesos nulos ou próximos de zero para _clusters_ desnecessários.

Basta definir o número de componentes para um valor que se acredita ser **maior do que o número ideal** de _clusters_, e o algoritmo eliminará os _clusters_ desnecessários automaticamente.

In [ ]:
from sklearn.mixture import BayesianGaussianMixture

# supomos um valor máximo de 10 para o número de clusters
bgm = BayesianGaussianMixture(random_state=42, n_components=10, n_init=20)
bgm.fit(X);

De fato, o algoritmo detectou automaticamente que apenas 3 componentes são necessários:

In [ ]:
print('pesos:', np.round(bgm.weights_, 2))

In [ ]:
# gráfico
plt.figure(figsize=(8, 4))
plot_gaussian_mixture(bgm, X)
plt.show()

# Limitações

Modelos de Mistura Gaussiana funcionam bem em aglomerados com formas elipsoidais, mas se for feito um ajuste em um conjunto de dados com formas diferentes, o resultado pode ser diferente do esperado.

Por exemplo, vamos ver o que acontece se usarmos o modelo de Mistura Gaussiana com aprendizado Bayesiano para agrupar o _dataset_ sintético `moons`.

In [ ]:
from sklearn.datasets import make_moons

X_moons, y_moons = make_moons(random_state=42, n_samples=1000, noise=0.05)

bgm = BayesianGaussianMixture(random_state=42, n_components=10, n_init=10)
bgm.fit(X_moons)

print('pesos:', np.round(bgm.weights_, 2))

In [ ]:
# gráfico do dataset (à esquerda) e do resultado (à direita)
plt.figure(figsize=(12, 4))
plt.subplot(121)
plt.plot(X_moons[:, 0], X_moons[:, 1], 'k.', markersize=2)
plt.xlabel('$x_1$', fontsize=14)
plt.ylabel('$x_2$', fontsize=14, rotation=0)
plt.subplot(122)
plot_gaussian_mixture(bgm, X_moons, show_ylabels=False)
plt.tight_layout()
plt.show()

Note que ao invés de detectar dois aglomerados em forma de lua, o algoritmo detectou 8 aglomerados elipsoidais.

Porém esta aproximação parece ser adequada quando examinamos as linhas de densidade. Dependendo da aplicação, podemos manter este modelo ou tentar outro algoritmo para este _dataset_.

# Demonstração: clusterização e detecção de anomalias em fotos de rostos

Vamos demonstrar agora as tarefas de clusterização e detecção de anomalias usando o conjunto de dados clássico **Olivetti Faces**.

Este _dataset_ contém 400 imagens de faces em tons de cinza, cada uma com resolução de 64 × 64 pixels. Cada imagem é achatada em um vetor de uma dimensão e comprimento 4096. Foram fotografadas 40 pessoas diferentes (10 vezes cada), e a tarefa usual é treinar um modelo que possa prever qual pessoa está representada em cada foto.

In [ ]:
from sklearn.datasets import fetch_olivetti_faces

olivetti = fetch_olivetti_faces()

X = olivetti.data
y = olivetti.target

In [ ]:
classes, quantidade = np.unique(olivetti.target, return_counts=True)
print('classes:', classes)
print('quantidade:', quantidade)

## Preparação dos dados com PCA

Como cada instância tem 4096 atributos, podemos tentar acelerar o aprendizado posterior aplicando agora uma tarefa de **redução de dimensionalidade** usando o algoritmo `PCA`.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.99)

X_pca = pca.fit_transform(X)

print('novo número de dimensões:', pca.n_components_)

## Agrupamento com Misturas Gaussianas

Aqui vamos comparar a diferença entre usar o algoritmo `BayesianGaussianMixture` (verificando quantos _clusters_ são detectados ao se impor um limite superior de 100 _clusters_) e o algoritmo `GaussianMixture`(com um número fixo de 40 _clusters_).

Lembrando, é preciso fazer o treino sobre a versão dos dados com dimensão reduzida, disponíveis em `X_pca`.

In [ ]:
from sklearn.mixture import BayesianGaussianMixture

# supomos um valor máximo de 100 para o número de clusters
bgm = BayesianGaussianMixture(random_state=42, n_components=100, n_init=10)
bgm.fit(X_pca);

print('pesos:', np.round(bgm.weights_, 2))

In [ ]:
from sklearn.mixture import GaussianMixture

# supomos um valor exato de 40 para o número de clusters
gm = GaussianMixture(random_state=42, n_components=40, n_init=10)
gm.fit(X_pca);

print('pesos:', np.round(gm.weights_, 2))

Agora vamos visualizar os 40 grupos de faces selecionados pelo último modelo, para verificar se a clusterização foi adequada.

In [ ]:
# função auxiliar
def plot_faces(faces, labels, n_cols=20):
    faces = faces.reshape(-1, 64, 64)
    n_rows = (len(faces) - 1) // n_cols + 1
    plt.figure(figsize=(n_cols, n_rows * 1.1))
    for index, (face, label) in enumerate(zip(faces, labels)):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(face, cmap='gray')
        plt.axis('off')
        plt.title(label)
    plt.show()

In [ ]:
# exibe todas as faces de cada cluster (mostrando o rótulo real acima de cada foto)

rótulos = gm.predict(X_pca)
# rótulos = bgm.predict(X_pca)

for cluster_id in np.unique(rótulos):
    print('cluster', cluster_id)
    in_cluster = rótulos == cluster_id
    faces = X[in_cluster]
    labels = y[in_cluster]
    plot_faces(faces, labels)

O resultado de fato agrupa faces bastante similares, mas nem sempre da mesma pessoa. Dependendo da aplicação, poderíamos precisar de mais clusters ou ainda usar _features_ adicionais para aumentar as chances de identificar a mesma pessoa em poses diferentes.

## Geração de novas instâncias

Usando o modelo `gm` treinado anteriormente, podemos gerar algumas novas faces com o método `.sample()`.

Note que novas instâncias têm o número reduzido de dimensões, então para visualizá-las precisaremos usar a transformação inversa do PCA, via método `.inverse_transform()`, para transformar essas instâncias sintéricas em imagens.

In [ ]:
n_gen_faces = 20
gen_faces_reduced, y_gen_faces = gm.sample(n_samples=n_gen_faces)
gen_faces = pca.inverse_transform(gen_faces_reduced)

In [ ]:
plot_faces(gen_faces, y_gen_faces, n_cols=10)

## Detecção de faces anômalas

Agora iremos criar novas imagens, fazendo um giro, invertendo ou escurecendo imagens originais.

Então podemos verificar se o modeloé capaz de detectar estas anomalias. Faremos isso comparando a saída do método `.score_samples()`, que retorna as densidades calculadas, para imagens normais e para as anomalias.

In [ ]:
# cria imagens rotacionadas
n_rotated = 4
rotated = np.transpose(X[:n_rotated].reshape(-1, 64, 64), axes=[0, 2, 1])
rotated = rotated.reshape(-1, 64*64)
y_rotated = y[:n_rotated]

# cria imagens invertidas verticalmente
n_flipped = 3
flipped = X[:n_flipped].reshape(-1, 64, 64)[:, ::-1]
flipped = flipped.reshape(-1, 64*64)
y_flipped = y[:n_flipped]

# cria imagens escurecidas
n_darkened = 3
darkened = X[:n_darkened].copy()
darkened[:, 1:-1] *= 0.3
y_darkened = y[:n_darkened]

X_bad_faces = np.r_[rotated, flipped, darkened]
y_bad = np.concatenate([y_rotated, y_flipped, y_darkened])

plot_faces(X_bad_faces, y_bad)

In [ ]:
# precisamo reduzir a dimensão destas novas imagens
X_bad_faces_pca = pca.transform(X_bad_faces)

In [ ]:
# agora veremos a densidade calculada para cada uma delas
gm.score_samples(X_bad_faces_pca)

As faces ruins são todas consideradas **altamente improváveis** pelo modelo de Mistura Gaussiana. Vamos comparar esses valores com as pontuações das instâncias originais:

In [ ]:
gm.score_samples(X_pca[:10])

## Detecção de anomalias usando redução de dimensionalidade

Para finalizar, vamos agora mostrar que algumas técnicas de redução de dimensionalidade também podem ser usadas para detecção de anomalias.

Aqui vamos usar o conjunto de dados Olivetti Faces, já reduzido por PCA e preservando 99% da variação.

A estratégia é a seguinte: calcular o **erro de reconstrução para cada imagem**. Vamos examinar as imagens modificadas construídas anteriormente e observar seu erro de reconstrução, que será maior do que para imagens normais.

In [ ]:
# erro de reconstrução usando Mean Squared Error (MSE)
def erro_de_reconstrução(pca, X_original):
    X_pca = pca.transform(X_original)
    X_reconstruido = pca.inverse_transform(X_pca)
    mse = np.square(X_reconstruido - X_original).mean(axis=-1)
    return mse

In [ ]:
# erro médio de reconstrução para todas as imagens ruins
erro_de_reconstrução(pca, X_bad_faces).mean()

In [ ]:
# erro médio de reconstrução para todas as imagens originais
erro_de_reconstrução(pca, X).mean()

Ao plotarmos uma imagem reconstruída com muito erro, veremos que de fato ela visualmente se afasta bastante de um rosto normal.

In [ ]:
# faces ruins feitas anteriormente
plot_faces(X_bad_faces, y_bad)

In [ ]:
# faces ruins reconstruidas
X_bad_faces_reconstruidas = pca.inverse_transform(X_bad_faces_pca)
plot_faces(X_bad_faces_reconstruidas, y_bad)